This script produces public urban green spaces prediction based on sattelite image and previously created model.

In [ ]:
!pip install segmentation_models
!pip install patchify

In [ ]:
# import libraries
from patchify import patchify, unpatchify
from keras.models import load_model
import segmentation_models as sm
import gdal, ogr, os, osr

import numpy as np
%matplotlib inline

from PIL import Image
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.metrics import MeanIoU

Segmentation Models: using `keras` framework.


In [ ]:
# Connecting to the Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load image file for a prediction city

In [ ]:
def image_file_to_array(city):  
  '''  read the satellite image file   '''
 
  image_file_path = os.path.join(dir, city, image_file_name)
  image = gdal.Open(image_file_path)
  image_array = image.ReadAsArray()
  image_array = np.transpose(image_array, [1, 2, 0])  # transpose the first and third axis

  return image_array

In [ ]:
# set the directory and file 
dir = #location of main folder
image_file_name = 'image_bgr_nir_ndvi_landcover_ndbi_ndwi.tif'

city =  "Washington" 

image_array = image_file_to_array(city)

In [ ]:
#chips 3 [2,3,4]
image_array = image_array[:,:,[2,3,4]]

In [ ]:
print(image_array.shape)

(2082, 1811, 3)


Load the prediction model

In [ ]:
model_test = load_model('model location', compile = False)

model_test.compile(optimizer=Adam(learning_rate = 1e-4), loss = sm.losses.binary_focal_dice_loss, 
              metrics=['accuracy', sm.metrics.IOUScore(threshold=0.5), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]) # use the same parameters as in original training!

Chip image and predict on each chip

In [ ]:
scaler = MinMaxScaler()
img_patches = patchify(image_array, (256, 256, image_array.shape[2]), step=256)
predicted_patches = []  # initialise output
for i in range(img_patches.shape[0]):
    for j in range(img_patches.shape[1]):
      single_patch_img = img_patches[i, j, 0, :, :, :]
      single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape)
      single_patch_img = np.expand_dims(single_patch_img, axis = 0)  # expand dimension to fit the shape of training data of loaded_model
      pred = model_test.predict(single_patch_img)  # make prediction on single patch
      pred_argmax = np.argmax(pred, axis = 3)  # get the max value in axis = 3, need to do this because we are using one-hot encoding
      pred_argmax = np.expand_dims(pred_argmax, axis = 3)[0, :, :, :]  # expand dimension to fit shape
      predicted_patches.append(pred_argmax)

In [ ]:
# turn list into array
predicted_patches = np.array(predicted_patches)

# reshape array for unpatchify
predicted_patches_reshaped = predicted_patches.reshape((img_patches.shape[0], img_patches.shape[1], 256, 256, 1))
predicted_patches_reshaped = predicted_patches_reshaped[:, :, :, :, 0]  # to fit shape for unpatchify


In [ ]:
# merge chips together (unpatchify)
reconstructed_image = unpatchify(predicted_patches_reshaped, (256*img_patches.shape[0], 256*img_patches.shape[1]))

In [ ]:
# save predicted array into tiff file
# function to turn array into tiff file with metadata
def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):
    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Byte)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(32636)  # set epsg you want
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

def main(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):
    reversed_arr = array #[::-1] # reverse array so the tif looks like the array
    array2raster(newRasterfn, rasterOrigin, pixelWidth, pixelHeight, reversed_arr) # convert array to raster


In [ ]:
image_file_path = os.path.join(dir, city, image_file_name)

In [ ]:
# get metadata of original image
image = gdal.Open(image_file_path)
geo_transform = image.GetGeoTransform()
originX = geo_transform[0]
originY = geo_transform[3]
pixelWidth = geo_transform[1]
pixelHeight = geo_transform[5]


In [ ]:
# save predicted array as tiff file
pred_file_name = 'pred_pugs.tif'

if __name__ == "__main__":
    rasterOrigin = (originX, originY)
    pixelWidth = pixelWidth
    pixelHeight = pixelHeight
    newRasterfn = os.path.join(dir, city, pred_file_name)  # file path you want to save
    main(newRasterfn, rasterOrigin, pixelWidth, pixelHeight, reconstructed_image)